In [4]:
try:
    import pandopt as pdo
except:
    import sys, os
    sys.path.append('/'.join(os.getcwd().split('/')[:-2]))
    import pandopt as pdo
import pandas as pd
from PandOpt import *
import numpy as np
import tqdm 
import pandas as pd
import numpy as np
import timeit
import functools

import ast
import inspdef {func_int_identifier}_loop(Z):
    n = Z.shape[0]
    res = np.zeros((n, 1))
    for i in nb.prange(n):
        res[i, 0] = temporary(Z[i, :])
    return resect
import types
import numpy as np
import numba as nb
import pandas as pd
import time
import logging
import functools
import copy
import sys
from typing import Callable, Type, Dict, Tuple, Any
from numpy.lib.stride_tricks import sliding_window_view
logger = logging.getLogger()
logger.setLevel(0)



df = pd.DataFrame(np.random.randn(10000, 4), columns=['A', 'B', 'C', 'D']).astype(np.float32)

In [27]:
df.groupby('E')

In [16]:
df['E'] = df['A'].apply(lambda x: 1 if x <-2 else 2 if x <-1 else 3 if x<0 else 4 if x<2 else 5)

In [25]:
%timeit df.loc[df.groupby('E').groups[1]]

413 µs ± 4.84 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
%timeit df.to_numpy()[df.groupby('E').groups[1]]

333 µs ± 8.11 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [28]:
df = df.set_index('C')

In [29]:
df.groupby('E').groups

{1: [0.6074827313423157, -0.23682086169719696, -0.7583650350570679, -0.27089250087738037, 0.4290613830089569, 0.24819545447826385, -2.4794089794158936, 0.06844813376665115, -0.19698843359947205, -0.4155316650867462, -1.1914106607437134, 0.6859724521636963, -2.073493719100952, 1.111675500869751, -1.3581395149230957, 0.08854194730520248, 0.020363595336675644, 0.1263808161020279, 1.566175103187561, 0.28795868158340454, 0.11065826565027237, -1.0327575206756592, 1.6044726371765137, -0.32373762130737305, -1.3724071979522705, -0.09387419372797012, -0.49082884192466736, 0.5802645087242126, 0.4747120440006256, -0.7681862711906433, -3.308753728866577, -1.22916841506958, 1.5650982856750488, 0.3566230833530426, 1.3245059251785278, 0.7789903879165649, 0.8563732504844666, 1.0916802883148193, -0.5606488585472107, 1.731178879737854, -0.6979461908340454, 0.3962887227535248, -1.8103595972061157, -1.0783528089523315, 1.3451440334320068, -0.8595927357673645, -0.31391966342926025, 0.051755499094724655, 0.0

In [4]:

# Example usage
def simple_start(z):
    x = (z['A'] + z['B']) / z['C']
    x += z['B'] * z['D']
    return x / z['B']

def harder_func(z):
    x = (z['A'] + z['B']) / z['C']
    if x > 0:
        return x / z['B']
    x += z['B'] * z['D']
    return x * z['B']

def harder2_func(z):
    x = (z['A'] + z['B']) / z['C']
    if (k:=z['A']-z['C']) > (j:=z['B']/z['D']):
        return x / k
    x *= j
    return x - k if k > z['C'] else x + k

def harder3_func(z):
    g=lambda a, b: a if abs(a) > abs(b) else - 2 * (b**(-a))
    x = (z['A'] + z['B']) / z['C']
    if (k:=z['A']-z['C']) > (j:=g(z['B'],z['D'])):
        return j / k
    x *= j
    return x - k if k > z['C'] else x + k

In [5]:

# Assume pandopt, simple_start, harder_func, harder2_func, harder3_func are defined
def agg_sum(z):
    return np.sum(z)

def agg_mean(z):
    return np.mean(z)

def agg_max(z):
    return np.max(z)

def agg_min(z):
    return np.min(z)

def agg_std(z):
    return np.std(z)



def measure_performance(df, func, test_type, window_size=3):
    # try:
    if test_type == "apply":
        operation = functools.partial(df.apply, func, axis=1)
    elif test_type == "rolling":
        operation = functools.partial(df.rolling(window=window_size).apply, func, raw=True)
    elif test_type == "groupby":
        operation = functools.partial(df.groupby('A').apply, func)
    else:
        raise ValueError("Invalid test type")

    start_time = timeit.default_timer()
    result = operation()
    elapsed_time = timeit.default_timer() - start_time
    return np.sum(result), elapsed_time, None
    # except Exception as e:
        # return None, None, str(e)

def run_tests(data_size, test_funcs, agg_funcs, test_types):
    results = {}
    total_tests = (data_size - 1) * len(test_funcs + agg_funcs) * len(test_types)
    progress_bar = tqdm.tqdm(total=total_tests, desc="Running Tests", ncols=100)

    for test_num in range(1, data_size):
        df_size = int(10**test_num)
        pandas_df = pd.DataFrame(np.random.randn(df_size, 4), columns=['A', 'B', 'C', 'D']).astype(np.float32)
        pandopt_df = pandopt(pandas_df)

        for func in test_funcs + agg_funcs:
            for test_type in test_types:
                pandas_checksum, pandas_time, pandas_error = measure_performance(pandas_df, func, test_type)
                pandopt_checksum, pandopt_time, pandopt_error = measure_performance(pandopt_df, func, test_type)

                key = f"Size: 10^{test_num}, Func: {func.__name__}, Test: {test_type}"
                results[key] = {
                    "Pandas Time (s)": pandas_time,
                    "Pandopt Time (s)": pandopt_time,
                    "Checksum Pandas": pandas_checksum,
                    "Checksum Pandopt": pandopt_checksum,
                    "Pandas Error": pandas_error,
                    "Pandopt Error": pandopt_error
                }

                progress_bar.update(1)

    progress_bar.close()
    return pd.DataFrame.from_dict(results, orient='index')

# Run the tests
test_functions = []#[simple_start, harder_func, harder2_func, harder3_func]
agg_functions = [agg_sum, agg_mean, agg_max, agg_min, agg_std]
test_types = ["apply", "rolling", "groupby"]

results_df = run_tests(data_size=5, test_funcs=test_functions, agg_funcs=agg_functions, test_types=test_types)
print(results_df)

Running Tests:   0%|                                                         | 0/60 [00:00<?, ?it/s]

ValueError: DataFrame constructor not properly called!

In [ ]:
df_size = int(10**5)
pandas_df = pd.DataFrame(np.random.randn(df_size, 4), columns=['A', 'B', 'C', 'D']).astype(np.float32)
pandopt_df = pandopt(pandas_df)
test_types = ["apply", "rolling", "groupby"]
measure_performance(pandas_df, simple_start, "apply",)

In [18]:
measure_performance(pandopt_df, simple_start, "apply",)

/home/remi/PhDWork/pandopt/PandOpt.py:215: FutureWarning: The behavior of pandopt.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return fn(self, *args, **kwargs)


(0    2185600.75
 dtype: float32,
 0.0009504860208835453,
 None)

In [1]:
import sys
import numpy as np
sys.path.append('/home/remi/PhDWork/pandopt')
import pandopt as pd

def agg_sum(x):
    return np.sum(x)

df = pd.DataFrame(np.random.randn(10000, 4), columns=['A', 'B', 'C', 'D'])
print(type(df))

<class 'pandopt.frames.DataFrame'>


In [2]:
print('APPLY')
print(df.apply(agg_sum, axis=1))
print(type(df.apply(agg_sum, axis=1)))
print(df.apply(agg_sum, axis=0))
print(type(df.apply(agg_sum, axis=0)))
print(df.apply(agg_sum))
print(type(df.apply(agg_sum)))
print(df.apply(agg_sum).apply(agg_sum))
print(type(df.apply(agg_sum).apply(agg_sum)))
print('\n\n')
print('SUM')
print(df.sum(axis=1))
print(type(df.sum(axis=1)))
print(df.sum(axis=0))
print(type(df.sum(axis=0)))
print(df.sum())
print(type(df.sum()))
print(df.sum().sum())
print(type(df.sum().sum()))

APPLY
             0
0    -4.680139
1     0.379731
2     2.071919
3    -0.553120
4    -3.062971
...        ...
9995 -4.978951
9996 -2.192341
9997  1.194534
9998  0.170787
9999  1.784289

[10000 rows x 1 columns]
<class 'pandopt.frames.DataFrame'>
            0
A  -29.927881
B  -75.823959
C -240.599146
D -219.154204
<class 'pandopt.frames.DataFrame'>
            0
A  -29.927881
B  -75.823959
C -240.599146
D -219.154204
<class 'pandopt.frames.DataFrame'>
           0
0 -565.50519
<class 'pandopt.frames.DataFrame'>



SUM
0      -4.680139
1       0.379731
2       2.071919
3      -0.553120
4      -3.062971
          ...   
9995   -4.978951
9996   -2.192341
9997    1.194534
9998    0.170787
9999    1.784289
Length: 10000, dtype: float64
<class 'pandas.core.series.Series'>
A    -29.927881
B    -75.823959
C   -240.599146
D   -219.154204
dtype: float64
<class 'pandas.core.series.Series'>
A    -29.927881
B    -75.823959
C   -240.599146
D   -219.154204
dtype: float64
<class 'pandas.core.series.S

In [3]:
df.rolling(5)

here
             A         B         C         D
0    -0.926101 -1.516807 -1.129789 -1.107442
1     1.222792  0.572325 -1.166444 -0.248942
2     0.405347  0.176174  0.126856  1.363542
3    -0.412873  0.227066  0.197526 -0.564839
4    -0.484383 -0.398157 -1.605032 -0.575399
...        ...       ...       ...       ...
9995 -1.643941 -1.598993 -1.455970 -0.280046
9996 -0.444016 -1.933455  0.130889  0.054241
9997  0.222636  1.032003 -1.266852  1.206747
9998  0.760347  0.620871 -0.434712 -0.775720
9999 -0.114830  0.370474  0.879388  0.649256

[10000 rows x 4 columns]
             A         B         C         D
0    -0.926101 -1.516807 -1.129789 -1.107442
1     1.222792  0.572325 -1.166444 -0.248942
2     0.405347  0.176174  0.126856  1.363542
3    -0.412873  0.227066  0.197526 -0.564839
4    -0.484383 -0.398157 -1.605032 -0.575399
...        ...       ...       ...       ...
9995 -1.643941 -1.598993 -1.455970 -0.280046
9996 -0.444016 -1.933455  0.130889  0.054241
9997  0.222636  1.032003

RollOpt [window=5,center=False,axis=0,method=single]

In [4]:
df.rolling(5).apply(agg_sum)

here
             A         B         C         D
0    -0.926101 -1.516807 -1.129789 -1.107442
1     1.222792  0.572325 -1.166444 -0.248942
2     0.405347  0.176174  0.126856  1.363542
3    -0.412873  0.227066  0.197526 -0.564839
4    -0.484383 -0.398157 -1.605032 -0.575399
...        ...       ...       ...       ...
9995 -1.643941 -1.598993 -1.455970 -0.280046
9996 -0.444016 -1.933455  0.130889  0.054241
9997  0.222636  1.032003 -1.266852  1.206747
9998  0.760347  0.620871 -0.434712 -0.775720
9999 -0.114830  0.370474  0.879388  0.649256

[10000 rows x 4 columns]
             A         B         C         D
0    -0.926101 -1.516807 -1.129789 -1.107442
1     1.222792  0.572325 -1.166444 -0.248942
2     0.405347  0.176174  0.126856  1.363542
3    -0.412873  0.227066  0.197526 -0.564839
4    -0.484383 -0.398157 -1.605032 -0.575399
...        ...       ...       ...       ...
9995 -1.643941 -1.598993 -1.455970 -0.280046
9996 -0.444016 -1.933455  0.130889  0.054241
9997  0.222636  1.032003

,A,B,C,D
4,-0.195218,-0.939398,-3.576883,-1.133081
5,1.863479,1.041853,-2.606025,-0.122979
6,-0.474910,-1.571922,-0.061762,0.300660
7,0.603822,-1.437506,0.079643,-0.777279
8,1.314286,-1.588112,-0.512090,-0.729223
...,...,...,...,...
9995,-1.980692,-2.381017,-0.170692,1.794020
9996,-2.935525,-4.594523,0.824805,0.777312
9997,-2.916480,-1.511276,-1.742075,2.408877
9998,-1.294322,-1.012199,-2.065208,1.161731


In [ ]:
df = pd.DataFrame(np.random.randn(100000, 4), columns=['A', 'B', 'C', 'D'])
type(df.rolling(10).apply(agg_sum))


In [14]:
type(df.mean(axis=1))

pandas.core.series.Series